In [1]:
import pandas as pd
import numpy as np
import math

pbp = pd.read_html('https://www.basketball-reference.com/boxscores/pbp/201711130LAC.html',header=1)[0]
print(pbp.shape)
pbp_raw = pbp.copy()

(506, 6)


In [2]:
awayteam = pbp.columns[1]
hometeam = pbp.columns[5]
pbp.columns = ['time', 'awayevents','awaypts','score','homepts','homeevents']

print(awayteam,hometeam)
pbp['awayteam'] = awayteam
pbp['hometeam'] = hometeam
events = pbp['awayevents']
events = events.fillna(pbp['homeevents'])
pbp['event'] = events
pbp['isawayevent'] = 1-pd.isnull(pbp['awayevents'])

pbp.head()



Philadelphia LA Clippers


,time,awayevents,awaypts,score,homepts,homeevents,awayteam,hometeam,event,isawayevent
0,12:00.0,Start of 1st quarter,NaN,NaN,NaN,NaN,Philadelphia,LA Clippers,Start of 1st quarter,1
1,12:00.0,Jump ball: D. Jordan vs. J. Embiid (D. Saric g...,NaN,NaN,NaN,NaN,Philadelphia,LA Clippers,Jump ball: D. Jordan vs. J. Embiid (D. Saric g...,1
2,11:44.0,J. Redick misses 3-pt shot from 25 ft,NaN,0-0,NaN,NaN,Philadelphia,LA Clippers,J. Redick misses 3-pt shot from 25 ft,1
3,11:41.0,NaN,NaN,0-0,NaN,Defensive rebound by D. Jordan,Philadelphia,LA Clippers,Defensive rebound by D. Jordan,0
4,11:33.0,NaN,NaN,0-3,3.0,A. Rivers makes 3-pt shot from 27 ft (assist b...,Philadelphia,LA Clippers,A. Rivers makes 3-pt shot from 27 ft (assist b...,0


In [3]:
#first need to replace scores at the beginning of games and the rows that just mark the end of quarters 
pbp['score'] = pbp['score'].replace(to_replace='Score',method='ffill')
pbp['score'] = pbp['score'].fillna(method='bfill')

#then we split these to the away team scores and home team scores
#awayscore,homescore = score
#print([len(x.split('-')) for x in pbp['score']])
score = [x.split('-') for x in pbp['score']]
awayscore,homescore = np.transpose(np.array(score))
awayscore = [int(x) for x in awayscore]
homescore = [int(x) for x in homescore]
pbp['awayscore'] = awayscore
pbp['homescore'] = homescore

#now drop the redundant variables
pbp2 = pbp.copy()
pbp = pbp.drop(['awayevents','awaypts','score','homepts','homeevents'], axis=1)
pbp.head()

,time,awayteam,hometeam,event,isawayevent,awayscore,homescore
0,12:00.0,Philadelphia,LA Clippers,Start of 1st quarter,1,0,0
1,12:00.0,Philadelphia,LA Clippers,Jump ball: D. Jordan vs. J. Embiid (D. Saric g...,1,0,0
2,11:44.0,Philadelphia,LA Clippers,J. Redick misses 3-pt shot from 25 ft,1,0,0
3,11:41.0,Philadelphia,LA Clippers,Defensive rebound by D. Jordan,0,0,0
4,11:33.0,Philadelphia,LA Clippers,A. Rivers makes 3-pt shot from 27 ft (assist b...,0,0,3


In [ ]:
#should create lots of variables, like:
#quarter (how to treat overtime?)
#isrebound
#reboundplayer
#isassist
#assistplayer
#shooter
#is3ptshot...

In [4]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.basketball-reference.com/leagues/NBA_2018_games-october.html'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')    
links = []
for ref in soup.find_all('a'):
    link = ref.get('href')
    if link.startswith('/boxscores/2'):
        links.append(link)
print(len(links))
links[0:2]

urls = []
for link in links:
    urls.append('https://www.basketball-reference.com' + link)
print(urls[0:2])


104
['https://www.basketball-reference.com/boxscores/201710170CLE.html', 'https://www.basketball-reference.com/boxscores/201710170GSW.html']
